In [3]:
!pip install yfinance pandas numpy schedule



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.6 MB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 0.8/1.6 MB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 1.0/1.6 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 1.6 MB/s eta 0:00:00
  Created wheel for multitasking: filename

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [4]:
!pip install yfinance pandas numpy schedule


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
import schedule
from datetime import datetime, timedelta
import json
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('trading.log'),
        logging.StreamHandler()
    ]
)

In [9]:
class LiveDataManager:
    def __init__(self, tickers):
        self.tickers = tickers
        self.latest_prices = {}

    def is_market_open(self):
        """
        Approximate US stock market hours check (UTC)
        """
        now = datetime.utcnow()
        weekday = now.weekday()  # Monday = 0

        # Closed on weekends
        if weekday >= 5:
            return False

        # US Market hours: 14:30–21:00 UTC
        market_open = now.replace(hour=14, minute=30, second=0, microsecond=0)
        market_close = now.replace(hour=21, minute=0, second=0, microsecond=0)

        return market_open <= now <= market_close

    def update_live_prices(self):
        logging.info("Fetching live prices...")

        data = yf.download(
            tickers=self.tickers,
            period="1d",
            interval="5m",
            progress=False
        )

        prices = {}
        for ticker in self.tickers:
            try:
                prices[ticker] = data[ticker]["Close"].dropna().iloc[-1]
            except Exception:
                logging.warning(f"No price data for {ticker}")

        self.latest_prices = prices
        return prices






    
       
      
       
         


In [20]:
class LiveDataManager:
   from datetime import datetime, timezone

class LiveDataManager:
    def __init__(self, tickers):
        self.tickers = tickers
        self.latest_prices = {}

    def is_market_open(self):
        """
        Approximate US stock market hours (UTC, timezone-aware)
        """
        now = datetime.now(timezone.utc)
        weekday = now.weekday()  # Monday = 0

        # Closed on weekends
        if weekday >= 5:
            return False

        market_open = now.replace(hour=14, minute=30, second=0, microsecond=0)
        market_close = now.replace(hour=21, minute=0, second=0, microsecond=0)

        return market_open <= now <= market_close

    def update_live_prices(self):
        logging.info("Fetching live prices...")

        data = yf.download(
            tickers=self.tickers,
            period="1d",
            interval="5m",
            progress=False
        )

        prices = {}
        for ticker in self.tickers:
            try:
                prices[ticker] = data[ticker]["Close"].dropna().iloc[-1]
            except Exception:
                logging.warning(f"No price data for {ticker}")

        self.latest_prices = prices
        return prices

     


In [14]:
class PortfolioManager:
    """Manages current portfolio state"""
    
    def __init__(self, initial_capital=100000):
        self.initial_capital = initial_capital
        self.cash = initial_capital
        self.positions = {}  # {ticker: shares}
        self.trade_history = []
        self.portfolio_value_history = []
        
    def get_current_positions(self):
        """Return current positions"""
        return self.positions.copy()
    
    def get_portfolio_value(self, current_prices):
        """Calculate total portfolio value"""
        position_value = sum(
            shares * current_prices.get(ticker, 0)
            for ticker, shares in self.positions.items()
        )
        total_value = self.cash + position_value
        return total_value
    
    def execute_trade(self, ticker, shares, price, timestamp):
        """
        Execute a trade (buy or sell)
        
        shares > 0: Buy
        shares < 0: Sell
        """
        cost = shares * price
        
        # Update cash
        self.cash -= cost
        
        # Update positions
        if ticker in self.positions:
            self.positions[ticker] += shares
        else:
            self.positions[ticker] = shares
        
        # Remove zero positions
        if self.positions.get(ticker, 0) == 0:
            del self.positions[ticker]
        
        # Record trade
        trade = {
            'timestamp': timestamp,
            'ticker': ticker,
            'shares': shares,
            'price': price,
            'cost': cost,
            'cash_after': self.cash
        }
        self.trade_history.append(trade)
        
        logging.info(f"{'BUY' if shares > 0 else 'SELL'} {abs(shares)} shares of {ticker} @ ${price:.2f}")
        
    def rebalance_portfolio(self, target_positions, current_prices):
        """
        Rebalance portfolio to match target positions
        
        target_positions: {ticker: target_weight (-1 to 1)}
        current_prices: {ticker: current_price}
        """
        timestamp = datetime.now()
        total_value = self.get_portfolio_value(current_prices)
        
        trades_executed = 0
        
        # Calculate target shares for each position
        target_shares = {}
        for ticker, weight in target_positions.items():
            if ticker not in current_prices or current_prices[ticker] == 0:
                continue
            
            target_value = total_value * weight
            target_shares[ticker] = int(target_value / current_prices[ticker])
        
        # Calculate trades needed
        all_tickers = set(self.positions.keys()) | set(target_shares.keys())
        
        for ticker in all_tickers:
            current_shares = self.positions.get(ticker, 0)
            target_share = target_shares.get(ticker, 0)
            
            shares_to_trade = target_share - current_shares
            
            if shares_to_trade != 0 and ticker in current_prices:
                self.execute_trade(
                    ticker=ticker,
                    shares=shares_to_trade,
                    price=current_prices[ticker],
                    timestamp=timestamp
                )
                trades_executed += 1
        
        logging.info(f"✓ Rebalancing complete: {trades_executed} trades executed")
        
        # Record portfolio value
        new_value = self.get_portfolio_value(current_prices)
        self.portfolio_value_history.append({
            'timestamp': timestamp,
            'value': new_value,
            'return': (new_value / self.initial_capital - 1) * 100
        })
    
    def save_state(self, filename='portfolio_state.json'):
        """Save portfolio state to file"""
        state = {
            'cash': self.cash,
            'positions': self.positions,
            'trade_history': self.trade_history,
            'portfolio_value_history': self.portfolio_value_history
        }
        
        with open(filename, 'w') as f:
            json.dump(state, f, indent=2, default=str)
        
        logging.info(f"✓ Portfolio state saved to {filename}")
    
    def load_state(self, filename='portfolio_state.json'):
        """Load portfolio state from file"""
        try:
            with open(filename, 'r') as f:
                state = json.load(f)
            
            self.cash = state['cash']
            self.positions = state['positions']
            self.trade_history = state['trade_history']
            self.portfolio_value_history = state['portfolio_value_history']
            
            logging.info(f"✓ Portfolio state loaded from {filename}")
        except FileNotFoundError:
            logging.warning(f"No saved state found at {filename}")


In [29]:
class LiveStrategyEngine:
    """Real-time strategy execution engine"""
    
    def __init__(self, tickers, initial_capital=100000, rebalance_frequency='monthly'):
        self.tickers = tickers
        self.data_manager = LiveDataManager(tickers)
        self.portfolio = PortfolioManager(initial_capital)
        self.rebalance_frequency = rebalance_frequency
        self.last_rebalance = None
        
    def calculate_signals(self):
        """Calculate trading signals based on current data"""
        logging.info("Calculating signals...")
        
        # Get historical data for factor calculation
        prices = self.data_manager.get_historical_data(period='1y')
        
        if prices is None or prices.empty:
            logging.error("No historical data available")
            return {}
        
        # Calculate momentum (simple 3-month for demo)
        momentum_3m = prices / prices.shift(63) - 1
        momentum_score = momentum_3m.iloc[-1]
        
        # Rank stocks
        ranked = momentum_score.rank(pct=True)
        
        # Generate signals
        signals = {}
        for ticker in self.tickers:
            if ticker not in ranked:
                continue
            
            rank = ranked[ticker]
            
            if rank >= 0.8:  # Top 20%
                signals[ticker] = 0.05  # 5% position
            elif rank <= 0.2:  # Bottom 20%
                signals[ticker] = -0.05  # -5% short position
            else:
                signals[ticker] = 0  # No position
        
        logging.info(f"✓ Signals calculated: {len([s for s in signals.values() if s != 0])} active positions")
        return signals
    
    def should_rebalance(self):
        """Check if it's time to rebalance"""
        if self.last_rebalance is None:
            return True
        
        now = datetime.now()
        
        if self.rebalance_frequency == 'daily':
            return True
        elif self.rebalance_frequency == 'weekly':
            days_since = (now - self.last_rebalance).days
            return days_since >= 7
        elif self.rebalance_frequency == 'monthly':
            # Rebalance on first trading day of month
            return now.month != self.last_rebalance.month
        
        return False
    
    def run_trading_cycle(self):
        """Execute one trading cycle"""
        logging.info("="*60)
        logging.info("RUNNING TRADING CYCLE")
        logging.info("="*60)
        
        # Check if market is open
        if not self.data_manager.is_market_open():
            logging.info("Market is closed. Skipping cycle.")
            return
        
        # Get current prices
        current_prices = self.data_manager.get_live_prices()
        
        # Log current portfolio value
        portfolio_value = self.portfolio.get_portfolio_value(current_prices)
        pnl = (portfolio_value / self.portfolio.initial_capital - 1) * 100
        logging.info(f"Portfolio Value: ${portfolio_value:,.2f} (PnL: {pnl:+.2f}%)")
        
        # Check if we should rebalance
        if self.should_rebalance():
            logging.info("Rebalancing portfolio...")
            
            # Calculate new signals
            signals = self.calculate_signals()
            
            # Execute rebalancing
            self.portfolio.rebalance_portfolio(signals, current_prices)
            
            self.last_rebalance = datetime.now()
        else:
            logging.info("No rebalancing needed at this time")
        
        # Save state
        self.portfolio.save_state()
        
        logging.info("="*60)
    
    def start_live_trading(self, check_interval_minutes=60):
        """
        Start live trading loop
        
        check_interval_minutes: How often to check for rebalancing
        """
        logging.info("="*60)
        logging.info("STARTING LIVE TRADING SYSTEM")
        logging.info("="*60)
        logging.info(f"Tickers: {len(self.tickers)}")
        logging.info(f"Initial Capital: ${self.portfolio.initial_capital:,.2f}")
        logging.info(f"Rebalance Frequency: {self.rebalance_frequency}")
        logging.info(f"Check Interval: {check_interval_minutes} minutes")
        logging.info("="*60)
        
        # Schedule trading cycle
        schedule.every(check_interval_minutes).minutes.do(self.run_trading_cycle)
        
        # Run first cycle immediately
        self.run_trading_cycle()
        
        # Main loop
        try:
            while True:
                schedule.run_pending()
                time.sleep(60)  # Check every minute
        except KeyboardInterrupt:
            logging.info("\n" + "="*60)
            logging.info("SHUTTING DOWN LIVE TRADING SYSTEM")
            logging.info("="*60)
            self.portfolio.save_state()
            
            # Print final summary
            current_prices = self.data_manager.get_live_prices()
            final_value = self.portfolio.get_portfolio_value(current_prices)
            final_pnl = (final_value / self.portfolio.initial_capital - 1) * 100
            
            logging.info(f"Final Portfolio Value: ${final_value:,.2f}")
            logging.info(f"Total PnL: {final_pnl:+.2f}%")
            logging.info(f"Total Trades: {len(self.portfolio.trade_history)}")
       

In [18]:
def run_live_paper_trading():
    """Run live paper trading system"""
    
    # Configuration
    tickers = [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'JPM',
        'V', 'JNJ', 'WMT', 'PG', 'MA', 'UNH', 'HD', 'BAC'
    ]
    
    initial_capital = 100000
    rebalance_frequency = 'monthly'  # 'daily', 'weekly', or 'monthly'
    check_interval = 60  # Check every 60 minutes
    
    # Initialize engine
    engine = LiveStrategyEngine(
        tickers=tickers,
        initial_capital=initial_capital,
        rebalance_frequency=rebalance_frequency
    )
    
    # Start live trading
    engine.start_live_trading(check_interval_minutes=check_interval)


if __name__ == "__main__":
    # For paper trading (no real money)
     run_live_paper_trading()
    
   

2026-01-09 15:36:53,617 - INFO - ============================================================
2026-01-09 15:36:53,618 - INFO - STARTING LIVE TRADING SYSTEM
2026-01-09 15:36:53,619 - INFO - ============================================================
2026-01-09 15:36:53,619 - INFO - Tickers: 16
2026-01-09 15:36:53,620 - INFO - Initial Capital: $100,000.00
2026-01-09 15:36:53,621 - INFO - Rebalance Frequency: monthly
2026-01-09 15:36:53,621 - INFO - Check Interval: 60 minutes
2026-01-09 15:36:53,622 - INFO - ============================================================
2026-01-09 15:36:53,622 - INFO - ============================================================
2026-01-09 15:36:53,623 - INFO - RUNNING TRADING CYCLE
2026-01-09 15:36:53,624 - INFO - ============================================================


AttributeError: 'LiveDataManager' object has no attribute 'is_market_open'

In [21]:
from datetime import datetime
print("UTC time:", datetime.utcnow())
print("Market open:", engine.data_manager.is_market_open())


UTC time: 2026-01-09 10:15:47.521130


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19524\3492712307.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("UTC time:", datetime.utcnow())


NameError: name 'engine' is not defined

In [22]:
tickers = [
    'AAPL','MSFT','GOOGL','AMZN','META','TSLA','NVDA','JPM',
    'V','JNJ','WMT','PG','MA','UNH','HD','BAC'
]

initial_capital = 100000
rebalance_frequency = 'monthly'

engine = LiveStrategyEngine(
    tickers=tickers,
    initial_capital=initial_capital,
    rebalance_frequency=rebalance_frequency
)


In [23]:
from datetime import datetime, timezone

print("UTC time:", datetime.now(timezone.utc))
print("Market open:", engine.data_manager.is_market_open())


UTC time: 2026-01-09 10:16:17.819652+00:00
Market open: False


In [24]:
import matplotlib.pyplot as plt

def plot_equity_curve(portfolio_manager):
    if not portfolio_manager.portfolio_value_history:
        print("No portfolio history to plot")
        return
    
    df = pd.DataFrame(portfolio_manager.portfolio_value_history)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    plt.figure(figsize=(12, 6))
    plt.plot(df['timestamp'], df['value'])
    plt.title("Equity Curve (Portfolio Value Over Time)")
    plt.xlabel("Time")
    plt.ylabel("Portfolio Value ($)")
    plt.grid(True)
    plt.show()


In [25]:
plot_equity_curve(engine.portfolio)


No portfolio history to plot


In [26]:
engine.last_rebalance = None
engine.run_trading_cycle()


2026-01-09 15:50:45,814 - INFO - ============================================================
2026-01-09 15:50:45,816 - INFO - RUNNING TRADING CYCLE
2026-01-09 15:50:45,817 - INFO - ============================================================
2026-01-09 15:50:45,818 - INFO - Market is closed. Skipping cycle.


In [27]:
plot_equity_curve(engine.portfolio)


No portfolio history to plot
